****************************************
# Conversion of Hardware-triggered Data
****************************************

First imports ...

In [1]:
import cait as ai

:::{note}
**Script Execution**

If cait is executed within a Python script rather that with IPython (e.g. Jupyter Notebooks), the main routine has to start with:

    if __name__ == '__main__':
    
The need for the explicit main routine specification is common for multithreading in Python.
:::

## Generate Test Data

In [2]:
test_data = ai.data.TestData(filepath='test_data/mock_001', duration=1800)

In [3]:
# ---------------------------------------------------------
# RDT
# ---------------------------------------------------------
test_data._generate_rdt_file()
dh = ai.DataHandler()
dh.checkout_rdt(path_rdt='test_data/mock_001.rdt', read_events=1, verb=True)

Rdt file written.
DataHandler Instance created.
#############################################################
EVENT NUMBER:  0
detector number (starting at 0):  0
number of coincident pulses in digitizer module:  0
module trigger counter (starts at 0, when TRA or WRITE starts):  1
channel trigger delay relative to time stamp [µs]:  0
absolute time [s] (computer time timeval.tv_sec):  1602879726
absolute time [us] (computer time timeval.tv_us):  0
Delay of channel trigger to testpulse [us]:  0
time stamp of module trigger low word (10 MHz clock, 0 @ START WRITE ):  0
time stamp of module trigger high word (10 MHz clock, 0 @ START WRITE ):  6
number of qdc events accumulated until digitizer trigger:  0
measuring hours (0 @ START WRITE):  0.0016666667070239782
accumulated dead time of channel [s] (0 @ START WRITE):  0.0
test pulse amplitude (0. for pulses, (0.,10.] for test pulses, >10. for control pulses):  0.10000000149011612
DAC output of control program (proportional to heater power):

In [4]:
# ---------------------------------------------------------
# CON
# ---------------------------------------------------------
test_data._generate_con_file()
dh = ai.DataHandler()
dh.checkout_con(path_con='test_data/mock_001.con', read_events=5)

Con file written.
DataHandler Instance created.
5 control pulses read from CON file.
 	detector_nmbr,	 	pulse_height, 	time_stamp_low, 	time_stamp_high, 	dead_time, 	mus_since_last_tp
1	0		6.07		30000000		0			0.0	[0]
2	1		4.09		30000000		0			0.0	[0]
3	0		5.56		120000000		0			0.0	[0]
4	1		3.9		120000000		0			0.0	[0]
5	0		6.26		210000000		0			0.0	[0]


In [5]:
# ---------------------------------------------------------
# PAR
# ---------------------------------------------------------
test_data._generate_par_file()
# test by looking at the text file!

Par file written.


In [6]:
# ---------------------------------------------------------
# ALL FOR A SECOND FILE
# ---------------------------------------------------------
test_data.update_filepath(file_path='test_data/mock_002')
test_data.generate(start_offset=1.5 * 3600, source='hw')

Rdt file written.
Con file written.
Par file written.


## Data Conversion

In [7]:
# Conversion from Rdt to HDF5

path_data = 'test_data/'
file_names = ['mock_001',
              'mock_002']

for file in file_names:
    # --------------------------------------------------
    # Convert Rdt to H5
    # --------------------------------------------------

    dh = ai.DataHandler(run='01',
                        module='Test',
                        channels=[0, 1],
                        record_length=16384,
                        sample_frequency=25000)
    
    dh.convert_dataset(path_rdt=path_data,
                       fname=file,
                       path_h5=path_data,
                       tpa_list=[0, 1, -1],
                       calc_mp=True,
                       calc_nps=True)

    # --------------------------------------------------
    # Include con file
    # --------------------------------------------------

    dh.include_con_file(path_con_file=path_data + file + '.con')

DataHandler Instance created.
Start converting.
READ EVENTS FROM RDT FILE.
Total Records in File:  800
Event Counts:  399
WORKING ON EVENTS WITH TPA = 0.
CREATE DATASET WITH EVENTS.
CALCULATE MAIN PARAMETERS.
WORKING ON EVENTS WITH TPA = -1.
CREATE DATASET WITH NOISE.
WORKING ON EVENTS WITH TPA > 0.
CREATE DATASET WITH TESTPULSES.
CALCULATE MP.
Hdf5 dataset created in  test_data/
Filepath and -name saved.
Accessing CON File...
200 Control Pulses for channel 0 in file.
CON File included.
DataHandler Instance created.
Start converting.
READ EVENTS FROM RDT FILE.
Total Records in File:  800
Event Counts:  399
WORKING ON EVENTS WITH TPA = 0.
CREATE DATASET WITH EVENTS.
CALCULATE MAIN PARAMETERS.
WORKING ON EVENTS WITH TPA = -1.
CREATE DATASET WITH NOISE.
WORKING ON EVENTS WITH TPA > 0.
CREATE DATASET WITH TESTPULSES.
CALCULATE MP.
Hdf5 dataset created in  test_data/
Filepath and -name saved.
Accessing CON File...
200 Control Pulses for channel 0 in file.
CON File included.


In [8]:
# merge the two files

ai.data.merge_h5_sets(path_h5_a=path_data + file_names[0] + '-P_Ch0-L_Ch1.h5', 
                      path_h5_b=path_data + file_names[1] + '-P_Ch0-L_Ch1.h5', 
                      path_h5_merged=path_data + 'test_001.h5', 
                      continue_hours=True,
                      keep_original_files=False)

--> MERGE GROUP: events.
SET: event.
creating ...
SET: mainpar.
creating ...
SET: true_ph.
SET: true_onset.
SET: of_ph.
SET: sev_fit_par.
SET: sev_fit_rms.
SET: hours.
creating ...
SET: labels.
SET: testpulseamplitude.
SET: time_s.
creating ...
SET: time_mus.
creating ...
SET: pulse_height.
--> MERGE GROUP: testpulses.
SET: event.
creating ...
SET: mainpar.
creating ...
SET: true_ph.
SET: true_onset.
SET: of_ph.
SET: sev_fit_par.
SET: sev_fit_rms.
SET: hours.
creating ...
SET: labels.
SET: testpulseamplitude.
creating ...
SET: time_s.
creating ...
SET: time_mus.
creating ...
SET: pulse_height.
--> MERGE GROUP: noise.
SET: event.
creating ...
SET: mainpar.
SET: true_ph.
SET: true_onset.
SET: of_ph.
SET: sev_fit_par.
SET: sev_fit_rms.
SET: hours.
creating ...
SET: labels.
SET: testpulseamplitude.
SET: time_s.
creating ...
SET: time_mus.
creating ...
SET: pulse_height.
--> MERGE GROUP: controlpulses.
SET: event.
SET: mainpar.
SET: true_ph.
SET: true_onset.
SET: of_ph.
SET: sev_fit_par.
SE